# Program that translates English to French

In [1]:
import pdb
import pickle
import string

import time

import gensim
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import scipy
import sklearn
from gensim.models import KeyedVectors
from nltk.corpus import stopwords, twitter_samples
from nltk.tokenize import TweetTokenizer

In [2]:
# download google news word2vec

!wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2021-06-26 11:02:50--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.147.118
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.147.118|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  41.5MB/s    in 40s     

2021-06-26 11:03:30 (39.7 MB/s) - ‘GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [3]:
# unzip the downloaded file
!gzip -d GoogleNews-vectors-negative300.bin.gz

In [4]:
# french embeddings
!curl -o ./wiki.multi.fr.vec https://dl.fbaipublicfiles.com/arrival/vectors/wiki.multi.fr.vec

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  599M  100  599M    0     0  50.8M      0  0:00:11  0:00:11 --:--:-- 55.9M


In [5]:
en_embeddings = KeyedVectors.load_word2vec_format('/content/GoogleNews-vectors-negative300.bin', binary=True)
fr_embeddings = KeyedVectors.load_word2vec_format('/content/wiki.multi.fr.vec')

## Utilities

In [6]:
def get_dict(filename):
  '''
    This function returns the english to french dictionary given a file where each column corresponds to a word.
  '''
  my_file = pd.read_csv(filename, sep=' ')
  etof = {}
  for i in range(len(my_file)):
    en = my_file.iloc[i,0]
    fr = my_file.iloc[i,1]
    etof[en] = fr
  return etof

def cosine_similarity(A,B):
  """
  Input: 
    A: a numpy array corresponds to a word vector
    B: a numpy array corresponds to a word vector
  Output: 
    cos: numerical number representing the cosine similarity between A and B
  """
  dot = np.dot(A, B)
  norma = np.linalg.norm(A)
  normb = np.linalg.norm(B)
  cos = dot / (norma * normb)
  return cos

## Loading the english to french dictionaries

In [7]:
en_fr_train = get_dict('/content/en-fr.train.txt')
print(f'The length of the english to french training dictionary is {len(en_fr_train)}')
en_fr_test = get_dict('/content/en-fr.test.txt')
print(f'The length of the english to french test dictionary is {len(en_fr_test)}')

The length of the english to french training dictionary is 5000
The length of the english to french test dictionary is 1500


In [8]:
english_set = set(en_embeddings.vocab)
french_set = set(fr_embeddings.vocab)
en_embeddings_subset = {}
fr_embeddings_subset = {}
french_words = set(en_fr_train.values())

for en_word in en_fr_train.keys():
  fr_word = en_fr_train[en_word]
  if fr_word in french_set and en_word in english_set:
    en_embeddings_subset[en_word] = en_embeddings[en_word]
    fr_embeddings_subset[fr_word] = fr_embeddings[fr_word]

for en_word in en_fr_test.keys():
  fr_word = en_fr_test[en_word]
  if fr_word in french_set and en_word in english_set:
    en_embeddings_subset[en_word] = en_embeddings[en_word]
    fr_embeddings_subset[fr_word] = fr_embeddings[fr_word]

In [9]:
# embedding dimensions
print(en_embeddings_subset['the'].shape[0])
print(fr_embeddings_subset['la'].shape[0])

300
300


## Building matrices X and Y

X: mxn matrix, where m = number of rows corresponding to m **english** words; n = embedding dimensions

Y: mxn matrix, where m = number of rows corresponding to m **french** words; n = embedding dimensions

In [10]:
def get_matrices(en_fr, french_vecs, english_vecs):
  """
  Input:
    en_fr: English to french dictionary
    french_vecs: French words to their corresponding word embeddings
    english_vecs: English words to their corresponding word embeddings
  Output: 
    X: a matrix where the columns are the English embeddings
    Y: a matrix where the columns are the French embeddings
    R: the projection matrix that minimizes the F norm ||XR - Y||^2
  """
  
  X_l = list()
  Y_l = list()

  english_set = set(english_vecs.keys())
  french_set = set(french_vecs.keys())

  french_words = set(en_fr.values())
  
  for en_word, fr_word in en_fr.items():
    if fr_word in french_set and en_word in english_set:
      # get the english embedding

      en_vec = english_vecs[en_word]

      # get the french embedding
      fr_vec = french_vecs[fr_word]

      # add the english embedding to the list
      X_l.append(en_vec)

      # add the frencg embedding to the list
      Y_l.append(fr_vec)

  X = np.stack(X_l, axis=0)
  Y = np.stack(Y_l, axis=0)

  return X,Y

In [11]:
# use get_matrices to obtain X_train and Y_train of english and french word embeddings into the corresponding vector space models
X_train, Y_train = get_matrices(en_fr_train, fr_embeddings_subset, en_embeddings_subset)

In [12]:
print(f'X_train shape: {X_train.shape}')
print(f'Y_train shape: {Y_train.shape}')

X_train shape: (4932, 300)
Y_train shape: (4932, 300)


## Translation as linear transformation of embeddings

Given dictionaries of English and French word embeddings you will create a transformation matrix `R`
* Given an English word embedding, $\mathbf{e}$, you can multiply $\mathbf{eR}$ to get a new word embedding $\mathbf{f}$.
    * Both $\mathbf{e}$ and $\mathbf{f}$ are row vectors
* You can then compute the nearest neighbors to `f` in the french embeddings and recommend the word that is most similar to the transformed word embedding.

### Describing translation as the minimization problem

Find a matrix `R` that minimizes the following equation. 

$$\arg \min _{\mathbf{R}}\| \mathbf{X R} - \mathbf{Y}\|_{F} $$

### Frobenius norm

The Frobenius norm of a matrix $A$ (assuming it is of dimension $m,n$) is defined as the square root of the sum of the absolute squares of its elements:

$$\|\mathbf{A}\|_{F} \equiv \sqrt{\sum_{i=1}^{m} \sum_{j=1}^{n}\left|a_{i j}\right|^{2}}$$

### Actual loss function
In the real world applications, the Frobenius norm loss:

$$\| \mathbf{XR} - \mathbf{Y}\|_{F}$$

is often replaced by it's squared value divided by $m$:

$$ \frac{1}{m} \|  \mathbf{X R} - \mathbf{Y} \|_{F}^{2}$$

where $m$ is the number of examples (rows in $\mathbf{X}$).

* The same R is found when using this loss function versus the original Frobenius norm.
* The reason for taking the square is that it's easier to compute the gradient of the squared Frobenius.
* The reason for dividing by $m$ is that we're more interested in the average loss per embedding than the  loss for the entire training set.
    * The loss for all training set increases with more words (training examples),
    so taking the average helps us to track the average loss regardless of the size of the training set.

#### Step 1: Computing the loss
* The loss function will be squared Frobenoius norm of the difference between
matrix and its approximation, divided by the number of training examples $m$.
* Its formula is:
$$ L(X, Y, R)=\frac{1}{m}\sum_{i=1}^{m} \sum_{j=1}^{n}\left( a_{i j} \right)^{2}$$

where $a_{i j}$ is value in $i$th row and $j$th column of the matrix $\mathbf{XR}-\mathbf{Y}$.

In [13]:
def compute_loss(X,Y,R):
  """
  Inputs:
    X: a matrix of dimension (mxn) where the columns are the english embeddings
    Y: a matrix of dimension (mxn) where the columns are the french embeddings
    R: a matrix of dimension (nxn) - transformation matrix from english to french vector space embeddings
  Outputs:
    loss: a scalar value representing loss
  """
  m = X.shape[0]

  # XR - Y
  diff = np.dot(X,R) - Y

  # element-wise square of the difference
  diff_squared = np.square(diff)

  # sum of the squared elements
  sum_diff_squared = np.sum(diff_squared)

  # divide by number of examples
  loss = sum_diff_squared / m

  return loss

### Step 2: Computing the gradient of loss in respect to transform matrix R

* Calculate the gradient of the loss with respect to transform matrix `R`.
* The gradient is a matrix that encodes how much a small change in `R`
affect the change in the loss function.
* The gradient gives us the direction in which we should decrease `R`
to minimize the loss.
* $m$ is the number of training examples (number of rows in $X$).
* The formula for the gradient of the loss function $𝐿(𝑋,𝑌,𝑅)$ is:

$$\frac{d}{dR}𝐿(𝑋,𝑌,𝑅)=\frac{d}{dR}\Big(\frac{1}{m}\| X R -Y\|_{F}^{2}\Big) = \frac{2}{m}X^{T} (X R - Y)$$

In [14]:
def compute_gradient(X,Y,R):
  """
  Inputs:
    X: a matrix of dimension (mxn) where the columns are the english embeddings
    Y: a matrix of dimension (mxn) where the columns are the french embeddings
    R: a matrix of dimension (nxn) - transformation matrix from english to french vector space embeddings
  Outputs:
    grad: a scalar value - gradient of the loss function L for given X,Y,R
  """
  m = X.shape[0]

  grad = X.T @ (X @ R - Y) * 2/m
  return grad

### Step 3: Finding the optimal R with gradient descent algorithm

In [15]:
def align_embeddings(X, Y, train_steps=100, learning_rate=0.0003):
    """"
    Inputs:
        X: a matrix of dimension (m,n) where the columns are the English embeddings.
        Y: a matrix of dimension (m,n) where the columns correspong to the French embeddings.
        train_steps: positive int - describes how many steps will gradient descent algorithm do.
        learning_rate: positive float - describes how big steps will  gradient descent algorithm do.
    Outputs:
        R: a matrix of dimension (n,n) - the projection matrix that minimizes the F norm ||X R -Y||^2
    """
    np.random.seed(129)

    R = np.random.rand(X.shape[1], X.shape[1])

    for i in range(train_steps):
        if i % 25 == 0:
            print(f"loss at iteration {i} is: {compute_loss(X, Y, R):.4f}")
        
        gradient = compute_gradient(X,Y,R)

        # update R by subtracting the learning rate times gradient
        R -= learning_rate * gradient
    return R

In [16]:
# Testing the function on random data
np.random.seed(129)
m = 10
n = 5
X = np.random.rand(m, n)
Y = np.random.rand(m, n) * .1
R = align_embeddings(X, Y)

loss at iteration 0 is: 3.7242
loss at iteration 25 is: 3.6283
loss at iteration 50 is: 3.5350
loss at iteration 75 is: 3.4442


## Calculate transformation matrix R

In [17]:
R_train = align_embeddings(X_train, Y_train, train_steps=400, learning_rate=0.8) 

loss at iteration 0 is: 963.0146
loss at iteration 25 is: 97.8292
loss at iteration 50 is: 26.8329
loss at iteration 75 is: 9.7893
loss at iteration 100 is: 4.3776
loss at iteration 125 is: 2.3281
loss at iteration 150 is: 1.4480
loss at iteration 175 is: 1.0338
loss at iteration 200 is: 0.8251
loss at iteration 225 is: 0.7145
loss at iteration 250 is: 0.6534
loss at iteration 275 is: 0.6185
loss at iteration 300 is: 0.5981
loss at iteration 325 is: 0.5858
loss at iteration 350 is: 0.5782
loss at iteration 375 is: 0.5735


## Testing the translation using k-Nearest Neighbors

### k-Nearest neighbors algorithm

[k-Nearest neighbors algorithm](https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm) 
* k-NN is a method which takes a vector as input and finds the other vectors in the dataset that are closest to it. 
* The 'k' is the number of "nearest neighbors" to find (e.g. k=2 finds the closest two neighbors).

### Searching for the translation embedding
Since we're approximating the translation function from English to French embeddings by a linear transformation matrix $\mathbf{R}$, most of the time we won't get the exact embedding of a French word when we transform embedding $\mathbf{e}$ of some particular English word into the French embedding space. 
* This is where $k$-NN becomes really useful! By using $1$-NN with $\mathbf{eR}$ as input, we can search for an embedding $\mathbf{f}$ (as a row) in the matrix $\mathbf{Y}$ which is the closest to the transformed vector $\mathbf{eR}$

### Cosine similarity
Cosine similarity between vectors $u$ and $v$ calculated as the cosine of the angle between them.
The formula is 

$$\cos(u,v)=\frac{u\cdot v}{\left\|u\right\|\left\|v\right\|}$$

* $\cos(u,v)$ = $1$ when $u$ and $v$ lie on the same line and have the same direction.
* $\cos(u,v)$ is $-1$ when they have exactly opposite directions.
* $\cos(u,v)$ is $0$ when the vectors are orthogonal (perpendicular) to each other.

#### Note: Distance and similarity are pretty much opposite things.
* We can obtain distance metric from cosine similarity, but the cosine similarity can't be used directly as the distance metric. 
* When the cosine similarity increases (towards $1$), the "distance" between the two vectors decreases (towards $0$). 
* We can define the cosine distance between $u$ and $v$ as
$$d_{\text{cos}}(u,v)=1-\cos(u,v)$$

In [53]:
def nearest_neighbor(v, candidates, k=1):
  """
  Input:
    v: the vector to find the nearest neighbor for
    candidates: a set of vectors where we find the neighbors
    k: top k nearest neighbors to find
  Output:
    k_idx: indices of the top k closest vectors in sorted form
  """
  similarity_l = []
  for row in candidates:
    # get the cosing similarity
    cos_similarity = cosine_similarity(v, row)
    similarity_l.append(cos_similarity)

  
  sorted_ids = np.argsort(similarity_l)[::-1]
  k_idx = sorted_ids[:k]
  k_idx = np.flip(k_idx)
  return k_idx

In [34]:
# Test the implementation:
v = np.array([1, 0, 1])
candidates = np.array([[1, 0, 5], [-2, 5, 3], [2, 0, 1], [6, -9, 5], [9, 9, 9]])
print(candidates[nearest_neighbor(v, candidates, 1)])

here [2]
here [2]
[[2 0 1]]


## Testing the translation and comuting its accuracy

In [60]:
def test_vocabulary(X, Y, R):
  """
  Input:
      X: a matrix where the columns are the English embeddings.
      Y: a matrix where the columns correspong to the French embeddings.
      R: the transform matrix which translates word embeddings from English to French word vector space.
  Output:
      accuracy: for the English to French capitals
  """
  pred = X @ R

  num_correct = 0
  for i in range(len(pred)):
    pred_idx = nearest_neighbor(pred[i], Y, k=1)
    if pred_idx == i:
      num_correct += 1
  accuracy = num_correct / len(pred)
  return accuracy

In [61]:
X_val, Y_val = get_matrices(en_fr_test, fr_embeddings_subset, en_embeddings_subset)

In [62]:
acc = test_vocabulary(X_val, Y_val, R_train)  # this might take a minute or two
print(f"accuracy on test set is {acc:.3f}")

accuracy on test set is 0.557
